In [96]:
from collections import OrderedDict

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from math import log10, sqrt
import glob
import urllib.request

from bs4 import BeautifulSoup
import tqdm



factory = StemmerFactory()
stemmer = factory.create_stemmer()
stopwords_factory = StopWordRemoverFactory()
stopwords = stopwords_factory.get_stop_words()

kata_kunci = "Ketersediaan Vaksin Covid"


In [97]:
def stateForQueryExpansion(kata_kunci):
    
    words = []
    thesaurus = {}
    words_result = ''

    for word in kata_kunci.split(" "):
            stemmed_word = stemmer.stem(word)
            if word not in stopwords and stemmed_word not in words:
                words.append(stemmed_word)

    print(words)
    words = list(set(words))
    for x in words:
        name = x
        data = {"q": name}
        encoded_data = urllib.parse.urlencode(data).encode("utf-8")
        content = urllib.request.urlopen("http://sinonimkata.com/search.php", encoded_data)
        soup = BeautifulSoup(content, 'html.parser')

        print(x)
        words_result += x+' '


        # print(soup.find_all('a', {'rel': 'nofollow'}))

        try:

            for x in soup.find('td', {'width': '90%'}).find_all('a', {'rel': 'nofollow'}):
                # print(x.get_text())
                words_result += x.get_text()+' '

            synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
            synonym = [x.gettext() for x in synonym]

            thesaurus[x] = [x] + synonym

        except:
            thesaurus[x] = [name]

    return words_result
    

In [98]:
query_expansion = stateForQueryExpansion(kata_kunci)
print(query_expansion)

['sedia', 'vaksin', 'covid']
covid
vaksin
sedia
covid vaksin sedia selesai siap bersedia cawis sudah telah semula tadinya sebenarnya sebetulnya seharusnya seyogianya 


In [99]:
def give_path(fld_path, kata_kunci):                             #give path of the folder containing all documents
    dic = {}
    dic['kata_kunci'] = kata_kunci
    file_names = glob.glob(fld_path)
    files_150 = file_names
    for file in files_150[:5]:
        name = file.split('/')[-1]
        with open(file, 'r', errors='ignore') as f:
            data = f.read()
        dic[name] = data
    return dic
    
path = 'dataklasifikasitxt/*.txt'
docs = give_path(path, query_expansion)
print("\nDocument Length")
print(docs)


Document Length
{'kata_kunci': 'covid vaksin sedia selesai siap bersedia cawis sudah telah semula tadinya sebenarnya sebetulnya seharusnya seyogianya ', 'doc_ibu_dan_anak_17.txt': 'Pandemi Belum Reda, Tetap Prioritaskan Kepentingan Anak!\nAnak-anak di masa pandemi juga memiliki kerentanan yang tinggi baik dari sisi kesehatan maupun kehidupan sosial.\nSelain tingkat infeksi dan kematian anak pada kasus COVID-19 di Indonesia tergolong tinggi, juga terdapat situasi riskan seperti meningkatnya jumlah pekerja anak, malnutrisi, dan putus sekolah.\nMari prioritaskan kesehatan, keamanan, dan keselamatan anak. Pastikan pula dalam segala aktivitas anak tetap #PakaiMasker #JagaJarak dan #CuciTanganPakaiSabun\nBaca panduan keselamatan anak masa pandemi di https://s.id/lindungi-anak', 'doc_guru_dan_siswa_16.txt': 'Pembelajaran Jarak Jauh (PJJ) Dapat Bantuan Kuota Internet\nBantuan kuota data internet diberikan untuk guru, siswa, dosen, dan mahasiswa, untuk menunjang Pembelajaran Jarak Jauh (PJJ) d

In [100]:
def get_list_of_word(list_of_document, stopword):
    list_of_word = []
    for sentence in list_of_document.values():
        for word in sentence.split(" "):
            stemmed_word = stemmer.stem(word)
            if word not in stopword and stemmed_word not in list_of_word:
                list_of_word.append(stemmed_word)
    return list_of_word

list_of_word = get_list_of_word(docs, stopwords)
print(len(list_of_word))


214


In [101]:
def create_term_frequency(list_of_word, length_of_document_with_kk):
    term_frequency = []
    for _ in range(length_of_document_with_kk):
        term_frequency.append(dict(zip(list_of_word, [0 for _ in range(len(list_of_word))])))
    return term_frequency


term_frequency = create_term_frequency(list_of_word, len(docs))
for index, sentence in enumerate(docs.values()):
    for word in stemmer.stem(sentence).split(" "):
        if word in term_frequency[index]:
            term_frequency[index][word] += 1
print(len(term_frequency))

6


In [102]:
def create_document_frequency(list_of_word):
    return dict(zip(list_of_word, [0 for _ in range(len(list_of_word))]))

document_frequency = create_document_frequency(list_of_word)
for index, sentence in enumerate(term_frequency):
    # print(index, sentence)
    if index > 0:
        for key, value in sentence.items():
            if value:
                document_frequency[key] += 1
print(len(document_frequency))

214


In [103]:
def get_d_df(length_of_document, document_frequency):
    d_df = {}
    for key, value in document_frequency.items():
        try:
            d_df[key] = round((length_of_document / value), 3)
        except:
            d_df[key] = 0

        print(f"{key} =  {length_of_document} / {value} = {d_df[key]}")
    return d_df

d_df = get_d_df(len(docs)-1, document_frequency)
# print(d_df)

covid =  5 / 0 = 0
vaksin =  5 / 2 = 2.5
sedia =  5 / 1 = 5.0
selesai =  5 / 0 = 0
siap =  5 / 0 = 0
cawis =  5 / 0 = 0
mula =  5 / 0 = 0
tadi =  5 / 0 = 0
benar =  5 / 0 = 0
yogia =  5 / 0 = 0
 =  5 / 0 = 0
pandemi =  5 / 4 = 1.25
belum =  5 / 1 = 5.0
reda =  5 / 1 = 5.0
tetap =  5 / 1 = 5.0
prioritas =  5 / 1 = 5.0
penting =  5 / 1 = 5.0
anak anak =  5 / 0 = 0
masa =  5 / 3 = 1.667
milik =  5 / 2 = 2.5
rentan =  5 / 1 = 5.0
tinggi =  5 / 2 = 2.5
baik =  5 / 2 = 2.5
sisi =  5 / 2 = 2.5
sehat =  5 / 1 = 5.0
maupun =  5 / 1 = 5.0
hidup =  5 / 1 = 5.0
sosial selain =  5 / 0 = 0
tingkat =  5 / 3 = 1.667
infeksi =  5 / 1 = 5.0
mati =  5 / 1 = 5.0
anak =  5 / 1 = 5.0
kasus =  5 / 1 = 5.0
covid-19 =  5 / 4 = 1.25
indonesia =  5 / 3 = 1.667
golong =  5 / 1 = 5.0
dapat =  5 / 4 = 1.25
situasi =  5 / 1 = 5.0
riskan =  5 / 1 = 5.0
jumlah =  5 / 1 = 5.0
kerja =  5 / 1 = 5.0
malnutrisi =  5 / 1 = 5.0
putus =  5 / 1 = 5.0
sekolah mari =  5 / 0 = 0
aman =  5 / 1 = 5.0
selamat =  5 / 1 = 5.0
pasti = 

In [104]:
def get_idf(d_df):
    idf = {}
    for key, value in d_df.items():
        try:
            idf[key] = round(log10(value), 3)
        except:
            idf[key] = 0

    return idf

idf = get_idf(d_df)
print(idf)

{'covid': 0, 'vaksin': 0.398, 'sedia': 0.699, 'selesai': 0, 'siap': 0, 'cawis': 0, 'mula': 0, 'tadi': 0, 'benar': 0, 'yogia': 0, '': 0, 'pandemi': 0.097, 'belum': 0.699, 'reda': 0.699, 'tetap': 0.699, 'prioritas': 0.699, 'penting': 0.699, 'anak anak': 0, 'masa': 0.222, 'milik': 0.398, 'rentan': 0.699, 'tinggi': 0.398, 'baik': 0.398, 'sisi': 0.398, 'sehat': 0.699, 'maupun': 0.699, 'hidup': 0.699, 'sosial selain': 0, 'tingkat': 0.222, 'infeksi': 0.699, 'mati': 0.699, 'anak': 0.699, 'kasus': 0.699, 'covid-19': 0.097, 'indonesia': 0.222, 'golong': 0.699, 'dapat': 0.097, 'situasi': 0.699, 'riskan': 0.699, 'jumlah': 0.699, 'kerja': 0.699, 'malnutrisi': 0.699, 'putus': 0.699, 'sekolah mari': 0, 'aman': 0.699, 'selamat': 0.699, 'pasti': 0.398, 'segala': 0.699, 'aktivitas': 0.699, 'pakaimasker': 0.699, 'jagajarak': 0.699, 'cucitanganpakaisabun baca': 0, 'pandu': 0.699, 'https s id lindungi-anak': 0, 'ajar': 0.398, 'jarak': 0.398, 'jauh': 0.398, 'pjj': 0.398, 'bantu': 0.398, 'kuota': 0.398, 'int

In [105]:
def get_w_q_t(term_frequency, idf):
    w_q_t = []
    frekuensi_max = 2
    for index, document in enumerate(term_frequency):
        w_q_t.append({})

        for key, value in document.items():
            if index == 0:
                w_q_t[index][key] = (value/frekuensi_max) * idf[key]
            else:
                w_q_t[index][key] = value * idf[key]

    return w_q_t
    
w_q_t = get_w_q_t(term_frequency, idf)
print(w_q_t)



[{'covid': 0.0, 'vaksin': 0.199, 'sedia': 0.699, 'selesai': 0.0, 'siap': 0.0, 'cawis': 0.0, 'mula': 0.0, 'tadi': 0.0, 'benar': 0.0, 'yogia': 0.0, '': 0.0, 'pandemi': 0.0, 'belum': 0.0, 'reda': 0.0, 'tetap': 0.0, 'prioritas': 0.0, 'penting': 0.0, 'anak anak': 0.0, 'masa': 0.0, 'milik': 0.0, 'rentan': 0.0, 'tinggi': 0.0, 'baik': 0.0, 'sisi': 0.0, 'sehat': 0.0, 'maupun': 0.0, 'hidup': 0.0, 'sosial selain': 0.0, 'tingkat': 0.0, 'infeksi': 0.0, 'mati': 0.0, 'anak': 0.0, 'kasus': 0.0, 'covid-19': 0.0, 'indonesia': 0.0, 'golong': 0.0, 'dapat': 0.0, 'situasi': 0.0, 'riskan': 0.0, 'jumlah': 0.0, 'kerja': 0.0, 'malnutrisi': 0.0, 'putus': 0.0, 'sekolah mari': 0.0, 'aman': 0.0, 'selamat': 0.0, 'pasti': 0.0, 'segala': 0.0, 'aktivitas': 0.0, 'pakaimasker': 0.0, 'jagajarak': 0.0, 'cucitanganpakaisabun baca': 0.0, 'pandu': 0.0, 'https s id lindungi-anak': 0.0, 'ajar': 0.0, 'jarak': 0.0, 'jauh': 0.0, 'pjj': 0.0, 'bantu': 0.0, 'kuota': 0.0, 'internet bantu': 0.0, 'data': 0.0, 'internet': 0.0, 'beri': 0.

In [106]:
def get_q_d(w_q_t):
    q_d = []
    for index, document in enumerate(w_q_t):
        q_d.append({})
        total = 0
        for key, value in document.items():
            q_d[index][key] = round(value ** 2, 3)
            total += q_d[index][key]
        q_d[index]["total"] = round(sqrt(total), 3)
    return q_d

q_d = get_q_d(w_q_t)
print("Q/D WQT dipangkatkn 2 dan diakar")
for x in q_d:
    print(x['total'])

Q/D WQT dipangkatkn 2 dan diakar
0.727
6.67
4.364
7.449
7.511
4.845


In [107]:
def get_dj_q(tfidf):
    dj_q = []
    for index, document in enumerate(tfidf):
        dj_q.append({})
        total = 0
        for key, value in document.items():
            if index > 0:
                dj_q[index][key] = value * tfidf[0][key]
                total += dj_q[index][key]
            dj_q[index]["total"] = round(sqrt(total), 3)
    return dj_q


print("\nPerhitungan Sum (tfidf * tfidf_query) atau dj.q")
sum_dj_q = get_dj_q(w_q_t)
for x in sum_dj_q:
    print(x['total'])


Perhitungan Sum (tfidf * tfidf_query) atau dj.q
0.0
0.0
0.0
1.238
0.745
0.0


In [108]:
def get_sum_of_qd(q_d):
    sum_qd = []
    for index, document in enumerate(q_d):
        sum_qd.append({})
        total = 0
        for key, value in document.items():
            if index > 0:
                sum_qd[index][key] = value * q_d[0][key]
                total += sum_qd[index][key]
            sum_qd[index]["total"] = round(sqrt(total), 3)
    return sum_qd

print("\nPerhitungan |dj|.|q| (jarak dokumen * jarak query)")
sum_qd = get_sum_of_qd(q_d)
for x in sum_qd:
    print(x['total'])


Perhitungan |dj|.|q| (jarak dokumen * jarak query)
0.0
2.202
1.781
2.585
2.402
1.877


In [109]:
def get_sim(sumtfidf, sumqd):
    sim = []
    for index, document in enumerate(sumtfidf):
        sim.append({})
        if index > 0 :
            # print(sumtfidf[index]['total'],sumqd[index]['total'])
            # sim[index] = round((sumtfidf[index]['total']/sumqd[index]['total'])*2,3) if sumqd[index]['total'] != 0 else 0
            sim[index] = round((sumtfidf[index]['total']/sumqd[index]['total']),3) if sumqd[index]['total'] != 0 else 0

    return  sim



In [110]:
kesimpulan = {}
print("Perhitungan dj.q / |dj|.|q|")
sim = get_sim(sum_dj_q, sum_qd)

i = 0
for x in docs.keys():
    # print(x)
    if x != 'kata_kunci':
        kesimpulan[x] = sim[i]
    i += 1

Perhitungan dj.q / |dj|.|q|


In [112]:
sorted_value = OrderedDict(sorted(kesimpulan.items(), key=lambda x: x[1], reverse=True))

i = 1
for x in sorted_value:
    if kesimpulan[x] >= 0 :
        print(i, x, kesimpulan[x])
    i += 1

1 doc_vaksin_21.txt 0.479
2 doc_vaksin_20.txt 0.31
3 doc_ibu_dan_anak_17.txt 0.0
4 doc_guru_dan_siswa_16.txt 0.0
5 doc_guru_dan_siswa_17.txt 0.0
